In [ ]:
import os
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import time # Opsional: untuk mengukur waktu

# --- Konfigurasi ---
# Nama folder input dan output sesuai permintaan Anda
NAMA_FOLDER_INPUT = 'input/indo'
NAMA_FOLDER_OUTPUT_RELATIF = 'output/sastrawi' # Path relatif untuk output
# --- Akhir Konfigurasi ---

# Dapatkan path direktori kerja saat ini (cocok untuk notebook/interactive)
base_dir = os.getcwd()
print(f"Menggunakan direktori kerja saat ini sebagai basis: {base_dir}") # Tambahkan ini untuk verifikasi
input_folder_path = os.path.join(base_dir, NAMA_FOLDER_INPUT)
# Menggunakan path relatif untuk output, digabungkan dengan base_dir
output_folder_path = os.path.join(base_dir, NAMA_FOLDER_OUTPUT_RELATIF)

# 1. Inisialisasi Stemmer Sastrawi
print("Menginisialisasi stemmer Sastrawi...")
factory = StemmerFactory()
stemmer = factory.create_stemmer()
print("Stemmer siap.")

# 2. Buat folder output (termasuk subfolder) jika belum ada
# os.makedirs() akan membuat folder induk ('output') jika diperlukan
if not os.path.exists(output_folder_path):
    # Membuat direktori secara rekursif
    os.makedirs(output_folder_path)
    print(f"Folder output dibuat (termasuk subfolder jika perlu): {output_folder_path}")
else:
    print(f"Folder output sudah ada: {output_folder_path}")

# 3. Proses setiap file dalam folder input
print(f"\nMemulai proses stemming dari folder: {input_folder_path}")
start_time = time.time() # Catat waktu mulai

try:
    if not os.path.isdir(input_folder_path):
        raise FileNotFoundError(f"Folder input '{input_folder_path}' tidak ditemukan atau bukan direktori.")

    list_file = os.listdir(input_folder_path)
    processed_files = 0
    skipped_files = 0

    if not list_file:
        print("Folder input kosong, tidak ada file yang diproses.")

    for filename in list_file:
        input_file_path = os.path.join(input_folder_path, filename)

        # Pastikan itu adalah file dan berakhiran .txt
        if os.path.isfile(input_file_path) and filename.lower().endswith('.txt'):
            output_file_path = os.path.join(output_folder_path, filename) # Nama file output sama

            print(f"  -> Memproses file: {filename}...")

            try:
                # Baca konten file input
                with open(input_file_path, 'r', encoding='utf-8', errors='ignore') as f_in:
                    original_text = f_in.read()

                # Lakukan stemming
                stemmed_text = stemmer.stem(original_text)

                # Tulis hasil stemming ke file output
                with open(output_file_path, 'w', encoding='utf-8') as f_out:
                    f_out.write(stemmed_text)

                processed_files += 1

            except Exception as e:
                print(f"     ERROR saat memproses file {filename}: {e}")
        else:
            # Memberi tahu jika ada item yang bukan file .txt di folder input
            if os.path.isfile(input_file_path):
                 print(f"  -> Melewati file non-txt: {filename}")
            else:
                 print(f"  -> Melewati item yang bukan file: {filename}")
            skipped_files += 1


    end_time = time.time() # Catat waktu selesai
    total_time = end_time - start_time

    print(f"\nProses stemming selesai.")
    print(f"Jumlah file .txt yang diproses: {processed_files}")
    if skipped_files > 0:
        print(f"Jumlah item non-txt atau subfolder yang dilewati: {skipped_files}")
    print(f"Hasil disimpan di folder: {output_folder_path}")
    print(f"Total waktu eksekusi: {total_time:.2f} detik")

except FileNotFoundError as fnf_error:
    print(f"ERROR: {fnf_error}")
    print("Pastikan nama folder input sudah benar dan folder tersebut ada di lokasi yang sama dengan skrip.")
except Exception as e:
    print(f"Terjadi kesalahan umum yang tidak terduga: {e}")

Menggunakan direktori kerja saat ini sebagai basis: /home/xerces/project/stemming-project
Menginisialisasi stemmer Sastrawi...
Stemmer siap.
Folder output sudah ada: /home/xerces/project/stemming-project/output/sastrawi

Memulai proses stemming dari folder: /home/xerces/project/stemming-project/input
  -> Memproses file: Kierkegaard_ A Brief Introduction_1.txt...

Proses stemming selesai.
Jumlah file .txt yang diproses: 1
Hasil disimpan di folder: /home/xerces/project/stemming-project/output/sastrawi
Total waktu eksekusi: 13.40 detik
